In [ ]:
import xlrd
import copy

import pandas as pd
import numpy as np

pd.options.display.max_columns = 99
pd.options.display.max_rows = 15

In [ ]:
import datetime
import locale
locale.setlocale(locale.LC_TIME, 'ru_RU.UTF-8')

def find_rows_by_col_val(col, value):
    rows = []
    for row in df.iloc:
        if row[col] == value:
            rows.append(row)
            
    return rows

def capitalize_and_strip_column(column_name):
    df[column_name] = df[column_name].str.strip().str.capitalize()

def format_str_to_date(date):
    if isinstance(date, datetime.datetime):
        return date
    
    date = str(date)
    
    if isinstance(date, str):
        if len(date.split()) == 1:
            try:
                return datetime.datetime.strptime(date, u'%d.%m.%Yг.')
            except:
                pass
            
            try:
                return datetime.datetime.strptime(date, u'%d%m%Y')
            except:
                pass
            
            try:
                return datetime.datetime.strptime(date, u'%d.%m.%Y.')
            except:
                pass

            return datetime.datetime.strptime(date.strip(), u'%d.%m.%Y')

        elif len(date.split()) == 3:
            try:
                return datetime.datetime.strptime(date, u'%d %m %Y')
            except:
                pass

            return datetime.datetime.strptime(date, u'%d %B %Y')
    
def format_date_col(column_name):
    for i, date in enumerate(df[column_name]):
        
        try:
            df[column_name][i] = format_str_to_date(date)
        except:
            pass
            
def parse_dates_to_list_col(column_name):
     for i, dates in enumerate(df[column_name]):
        l = []
        
        try:
            date = format_str_to_date(dates)
            df[column_name][i] = [date.strftime('%Y-%m-%d %H:%M:%S')]
            continue
        except:
            pass
            
        if isinstance(dates, str):
            for date in dates.split():
                try:
                    date = format_str_to_date(date)
                    l.append(date.strftime('%Y-%m-%d %H:%M:%S'))
                except:
                    pass
                    
            df[column_name][i] = l
        elif not pd.isna(df[column_name][i]):
            df[column_name][i] = [df[column_name][i].strftime('%Y-%m-%d %H:%M:%S')]

vacines = [
    'Бешенство',
    'Мультикан-6',
    'Рабикан',
    'Нобивак Трикат',
    'Леоминор',
    'Пуревакс FelV',
    'Мультикан',
    'Мультифел-4',
    'Мультикан-4',
    'Мультикан-8',
    'Мультикан-9',
    'Nobivac Tricat Trio+R',
    'Нобивак DHPPI',
    'Нобивак Lepto',
    'Астерион DHPPi-L',
    'Нобивак Трикат Трио',
    'Нобивак Трикат Трио Леоминор',
    'Леоминор',
    'Нобивак',
    'Lepto',
    'Астерион']

drugs = [
    'Азинокс',
    'Паразицид-суспензия',
    'Стронгхолд',
    'Стронхолд',
    'Блох нэт/ альвет',
    'Тронцил',
    'Инсектал',
    'Дана СПОТ-ОН/Алевит',
    'Рольф Клуб',
    'Прател',
    'БАРС',
    'Рольф клуб 3D',
    'Инспектор Тотал К',
    'Празител',
    'Каниквантел Барс',
    'Дронтал',
    'Барс для кошек']

vacines.sort(key=lambda x: len(x), reverse=True)
drugs.sort(key=lambda x: len(x), reverse=True)

vacines_dict = dict([(vacines[i], str(i+1)) for i in range(len(vacines))])
drugs_dict = dict([(drugs[i], str(i+1)) for i in range(len(drugs))])

In [3]:
def prepare_dataframe():
    df.columns = ['index', 'animal__cart_number', 'animal__@kind', 'animal__age', 
              'animal__weight', 'animal__name', 'animal__@sex', 
              'animal__@breed', 'animal__@color', 'animal__@wool', 'animal__@ears', 
              'animal__@tail', 'animal__size', 'animal__special_signs', 
              
              'animal_shelter__aviary_number', 'animal__identification_number', 
              'animal__sterilization_status', 'animal__sterilization_veterinarian_name', 'animal__is_socialization',
              'animal_capture__certificater', 'animal_capture__certificater_date',
              'animal_capture__district', 'animal_capture__act', 'animal_capture__address',
             
              'owner_entity__organization_name', 'owner_entity__address', 'owner_entity__contacts',
              'owner_entity__owner_name', 'owner_entity__owner_address', 'owner_entity__owner_contact',
              'owner_individual__name', 'owner_individual__passport_series', 'owner_individual__issued', 
              'owner_individual__passport_date', 'owner_individual__passport_address', 'owner_individual__contact',
              'animal_shelter__arrived_date', 'animal_shelter__arrived_act', 
              'animal_shelter__leave_date', 'animal_shelter__leave_reason', 'animal_shelter__leave_act',
              'shelter__address', 'shelter__company', 'shelter__leader', 
              'animal_schelter__responsible', 
              'animal_drug__numbers', 'animal_drug__dates', 'animal_drug__names', 'animal_drug__doses',
              'animal_vacine__numbers', 'animal_vacine__dates', 'animal_vacine__names',
              'animal_vacine__series', 'animal_inspection__date', 'animal_inspection__anamnes']
    
    df.animal_vacine__names = df.animal_vacine__names.astype('str').str.lower()

    for vacine in vacines:
        df.animal_vacine__names = df.animal_vacine__names.str.replace(vacine.lower(), vacines_dict[vacine])

    for i in range(df.shape[0]):
        df.animal_vacine__names[i] = df.animal_vacine__names[i].replace('nobivac tricat trio+r', '2')

    df.animal_drug__names = df.animal_drug__names.astype('str').str.lower()

    for drug in drugs:
        df.animal_drug__names = df.animal_drug__names.str.replace(drug.lower(), drugs_dict[drug])

    for field in ['animal__@kind', 'animal__@sex', 'animal__@breed', 'animal__@wool', 
                  'animal__@tail', 'animal__@ears', 'animal__size', 'animal__special_signs',
                  'animal__is_socialization', 'animal_shelter__leave_reason']:

        capitalize_and_strip_column(field)

    for field in ['animal__sterilization_status', 'animal_shelter__arrived_date', 'animal_shelter__leave_date',
                  'animal_capture__certificater_date', 'owner_individual__passport_date', 'animal_inspection__date']:
        format_date_col(field)

    for field in ['animal_vacine__numbers', 'animal_vacine__series', 'animal_drug__numbers',
                  'animal_vacine__names', 'animal_drug__names', 'animal_drug__doses']:
        df[field] = df[field].astype('str').str.split()

    for field in ['animal_drug__dates', 'animal_vacine__dates']:
        parse_dates_to_list_col(field)

    df.animal__age = df.animal__age.astype('str').str.split('-').str[0]
    
    df.animal__is_socialization = df.animal__is_socialization.map({'Да': True, 'Нет': False})
    df.animal__size = df.animal__size.map({'Малый': '1', 'Средний': '2', 'Большой': '3', 'Крупный': '4'})

In [4]:
df = pd.read_excel('Data_set.xlsx')[:-1]
prepare_dataframe()

<ipython-input-3-373228ec3f3b>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.animal_vacine__names[i] = df.animal_vacine__names[i].replace('nobivac tricat trio+r', '2')
<ipython-input-2-a3857aab9faf>:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name][i] = format_str_to_date(date)
<ipython-input-2-a3857aab9faf>:63: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column_name][i] = [date.strftime('%Y-%m-%d %H:%M:%S')

In [5]:
df.drop('index', axis=1).to_csv('animal_data_set.csv', sep=';')

In [18]:
tails = [(str(ind+1), key) for ind, key in enumerate(df.animal__tail.value_counts().index)]
tails

[('1', 'Обычный'),
 ('2', 'Саблевидный'),
 ('3', 'Купированный'),
 ('4', 'Крючком'),
 ('5', 'Прутом'),
 ('6', 'Поленом')]

In [19]:
breeds = [(str(ind+1), key) for ind, key in enumerate(df.animal__breed.value_counts().index)]
breeds

[('1', 'Метис'), ('2', 'Алабай')]

In [ ]:
breeds = [(str(ind+1), key) for ind, key in enumerate(df.animal__breed.value_counts().index)]
breeds

In [20]:
color = [(str(ind+1), key) for ind, key in enumerate(df.animal__color.value_counts().index)]
color

[('1', 'чепрачный'),
 ('2', 'светло-коричневый'),
 ('3', 'черный'),
 ('4', 'черно-белый'),
 ('5', 'биколор'),
 ('6', 'рыжий'),
 ('7', 'тигровый'),
 ('8', 'белый'),
 ('9', 'триколор'),
 ('10', 'темно-коричневый'),
 ('11', 'палевый'),
 ('12', 'кремовый'),
 ('13', 'серебристый'),
 ('14', 'перец с солью'),
 ('15', 'черный с белым'),
 ('16', 'красный'),
 ('17', 'черепаховый'),
 ('18', 'соболиный'),
 ('19', 'голубой с белым'),
 ('20', 'шоколадный'),
 ('21', 'дымчатый'),
 ('22', 'золотой'),
 ('23', 'арлекин'),
 ('24', 'фавн (бежевый)'),
 ('25', 'черно-красный-белый'),
 ('26', 'красный с белым'),
 ('27', 'абркосовый'),
 ('28', 'мраморный'),
 ('29', 'голубо-кремовый черепаховый')]

In [6]:
df.animal_shelter__leave_reason.value_counts()

Передача в собственность (под опеку)    68
Смерть                                  31
Name: animal_shelter__leave_reason, dtype: int64

In [21]:
wool = [(str(ind+1), key) for ind, key in enumerate(df.animal__wool.value_counts().index)]
wool

[('1', 'Короткая'), ('2', 'Обычная'), ('3', 'Длинная'), ('4', 'Гладкая')]

In [22]:
ears = [(str(ind+1), key) for ind, key in enumerate(df.animal__ears.value_counts().index)]
ears

[('1', 'Стоячие'),
 ('2', 'Полустоячие'),
 ('3', 'Висячие'),
 ('4', 'Купированные')]

In [23]:
size = [(str(ind+1), key) for ind, key in enumerate(df.animal__size.value_counts().index)]
size

[('1', 'Средний'), ('2', 'Малый'), ('3', 'Крупный'), ('4', 'Большой')]

In [13]:
df.animal__tail.value_counts()

Обычный         225
Саблевидный      13
Купированный      1
Крючком           1
Прутом            1
Поленом           1
Name: animal__tail, dtype: int64

In [1326]:
df.animal__identification_number.value_counts()

643090000460843    2
643115161713764    2
900115001458175    1
643093300135061    1
900115001457834    1
                  ..
900115001364916    1
900115001364973    1
643090000457534    1
643090000457533    1
900115001458176    1
Name: animal__identification_number, Length: 240, dtype: int64

In [91]:
row = {}

for col in df.columns:
    if len(col.split('__')) != 2:
        continue 
        
    if not col.split('__')[0] in row:
        row[col.split('__')[0]] = [col.split('__')[1]]
    else:
        row[col.split('__')[0]].append(col.split('__')[1])

In [92]:
row.keys()

dict_keys(['animal', 'animal_shelter', 'animal_capture', 'owner_entity', 'owner_individual', 'shelter', 'animal_schelter', 'animal_drug', 'animal_vacine', 'animal_inspection'])

In [1031]:
for x in row['animal_shelter']:
    print(x)

arrived_date
arrived_act
leave_date
leave_reason
leave_act
